In [1]:
import random
import pandas as pd
import numpy as np
import sympy as sp
from scipy.optimize import minimize

In [13]:
# Define the symbolic variables
Sigma, Alpha = sp.symbols('Sigma Alpha', real=True)

# Define the terms for the inequalities
left_term_1 = 5**(1 - Sigma)
left_term_2 = sp.exp(-(-sp.log(0.3))**Alpha) * (20**(1 - Sigma) - 5**(1 - Sigma))
right_term_1 = 2**(1 - Sigma)
left_term_1_ineq3 = 15**(1 - Sigma)
left_term_2_ineq3 = sp.exp(-(-sp.log(0.9))**Alpha) * (20**(1 - Sigma) - 15**(1 - Sigma))

# Series values as defined in your problem statement
series_1_right2 = [34, 37, 41, 46, 53, 62, 75, 92, 110, 150, 200, 300, 500, 850]
series_2_right2 = [27, 28, 29, 30, 31, 32, 34, 36, 38, 41, 45, 50, 55, 65]

# Create matrices to store results
sigma_results = np.full((14, 14), "No Data", dtype=object)
alpha_results = np.full((14, 14), "No Data", dtype=object)

# Set grid
sigma_range = np.linspace()
alpha_range = np.linspace()
sigma_mesh, alpha_mesh = np.meshgrid(sigma_range, alpha_range)

# Convert symbolic expressions to numerical functions using lambdify
inequalities = []
for i in range(1, len(series_1_right2)):
    for j in range(1, len(series_2_right2)):
        # Define inequalities based on your series values
        right_term_2_ineq1 = sp.exp(-(-sp.log(0.1))**Alpha) * (series_1_right2[i]**(1 - Sigma) - 2**(1 - Sigma))
        right_term_2_ineq2 = sp.exp(-(-sp.log(0.1))**Alpha) * (series_1_right2[i-1]**(1 - Sigma) - 2**(1 - Sigma))
        right_term_2_ineq3 = sp.exp(-(-sp.log(0.7))**Alpha) * (series_2_right2[j-1]**(1 - Sigma) - 2**(1 - Sigma))
        right_term_2_ineq4 = sp.exp(-(-sp.log(0.7))**Alpha) * (series_2_right2[j]**(1 - Sigma) - 2**(1 - Sigma))

        # Simplify the expressions to reduce computation time
        ineq1 = sp.simplify(-left_term_1 - left_term_2 + (right_term_1 + right_term_2_ineq1))
        ineq2 = sp.simplify(left_term_1 + left_term_2 - (right_term_1 + right_term_2_ineq2))
        ineq3 = sp.simplify(left_term_1_ineq3 + left_term_2_ineq3 - (right_term_1 + right_term_2_ineq3))
        ineq4 = sp.simplify(-left_term_1_ineq3 - left_term_2_ineq3 + (right_term_1 + right_term_2_ineq4))
        
        inequality_funcs = [sp.lambdify((Sigma, Alpha), ineq, "numpy") for ineq in [ineq1, ineq2, ineq3, ineq4]]

        # Evaluate inequalities over the meshgrid
        overlap = np.ones(sigma_mesh.shape, dtype=bool)
        for func in inequality_funcs:
            result = func(sigma_mesh, alpha_mesh) > 0
            overlap &= result

        # Extract valid ranges from the overlap
        valid_sigma = sigma_mesh[overlap]
        valid_alpha = alpha_mesh[overlap]

        if valid_sigma.size > 0 and valid_alpha.size > 0:
            sigma_min, sigma_max = np.min(valid_sigma), np.max(valid_sigma)
            alpha_min, alpha_max = np.min(valid_alpha), np.max(valid_alpha)
            sigma_results[i][j] = f"{sigma_min:.2f}, {sigma_max:.2f}"
            alpha_results[i][j] = f"{alpha_min:.2f}, {alpha_max:.2f}"

# Convert results to DataFrames
sigma_df = pd.DataFrame(sigma_results)
alpha_df = pd.DataFrame(alpha_results)

# Export to CSV
sigma_df.to_csv('sigma_results.csv', index=False, header=False)
alpha_df.to_csv('alpha_results.csv', index=False, header=False)
